In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../execute'))

In [3]:
import json
import parcels
from datetime import timedelta
import kernels
import logging
import time
from importlib import reload  # Python 3.4+
from fieldset_vars import FieldsetVariable


In [20]:
import execute_PARCELS as exP

In [25]:
reload(exP)
reload(kernels)

<module 'kernels' from '/home/colin_beyers/lagrangian_drifts/execute/kernels.py'>

In [6]:
config = exP.load_config(config_path='../execute/config.json')

2024-10-01 16:22:16,350 - INFO - Loading configuration from ../execute/config.json


In [7]:
config

{'pathname': '/home/colin_beyers/data/cllc-U,V,mag.nc4',
 'variables': {'U': 'U', 'V': 'V'},
 'dimensions': {'lat': 'lat', 'lon': 'lon', 'time': 'time'},
 'particles': {'lon': [-135], 'lat': [40.0]},
 'kernels': ['AdvectionRK4', 'TotalDistance', 'BoundaryCheck'],
 'output': {'directory': '../output',
  'file_name': 'cllc-total_eulerian-TEST.zarr'},
 'runtime': {'days': 5, 'dt_minutes': 5},
 'output_interval_hours': 6}

In [8]:
kernel_list = exP.load_kernels(config.get("kernels", []), parcels, kernels)

2024-10-01 16:22:17,138 - INFO - Loaded kernels: ['AdvectionRK4', 'TotalDistance', 'BoundaryCheck']


In [9]:
kernel_list

[<function parcels.application_kernels.advection.AdvectionRK4(particle, fieldset, time)>,
 <function kernels.TotalDistance(particle, fieldset, time)>,
 <function kernels.BoundaryCheck(particle, fieldset, time)>]

In [10]:
extra_vars = exP.collect_extra_vars(kernel_list)

2024-10-01 16:22:17,991 - INFO - Collected extra variables for kernels: [PVar<distance|<class 'numpy.float32'>>, PVar<prev_lon|<class 'numpy.float32'>>, PVar<prev_lat|<class 'numpy.float32'>>, <FieldsetVariable lat_min>, <FieldsetVariable lat_max>, <FieldsetVariable lon_min>, <FieldsetVariable lon_max>]


In [11]:
extra_vars[3].value

<bound method FieldsetVariable.value of <FieldsetVariable lat_min>>

In [12]:
extra_particle_vars = []

In [13]:
for extra_var in extra_vars:
    if type(extra_var) == parcels.particle.Variable:
        print(extra_var)
        extra_particle_vars.append(extra_var)
particles = parcels.JITParticle.add_variables(extra_particle_vars)

PVar<distance|<class 'numpy.float32'>>
PVar<prev_lon|<class 'numpy.float32'>>
PVar<prev_lat|<class 'numpy.float32'>>


In [14]:
particles

parcels.particle.ScipyParticle.add_variable.<locals>.NewParticle

In [15]:
fieldset = exP.create_fieldset(config)

2024-10-01 16:22:22,574 - INFO - Creating FieldSet with filenames: {'U': '/home/colin_beyers/data/cllc-U,V,mag.nc4', 'V': '/home/colin_beyers/data/cllc-U,V,mag.nc4'}


In [16]:
for extra_var in extra_vars:
    if isinstance(extra_var, FieldsetVariable):
        fieldset.add_constant(extra_var.name, extra_var.value(fieldset))  # Pass fieldset here
    else:
        fieldset.add_constant(extra_var.name, extra_var)

In [17]:
print("Available attributes in fieldset:")
print(dir(fieldset))

Available attributes in fieldset:
['U', 'UV', 'V', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_UVfield', 'add_constant', 'add_constant_field', 'add_field', 'add_periodic_halo', 'add_vector_field', 'check_complete', 'checkvaliddimensionsdict', 'completed', 'computeTimeChunk', 'compute_on_defer', 'distance', 'from_a_grid_dataset', 'from_b_grid_dataset', 'from_c_grid_dataset', 'from_data', 'from_mitgcm', 'from_modulefile', 'from_mom5', 'from_nemo', 'from_netcdf', 'from_parcels', 'from_pop', 'from_xarray_dataset', 'get_fields', 'gridset', 'lat_max', 'lat_min', 'lon_max', 'lon_min', 'parse_wildcards', 'particlefile', 'prev_lat', 'prev_lon', 'time_origin', 'write']


In [18]:
# Accessing the individual attributes directly
print("Fieldset boundaries:")
print(f"lat_min: {fieldset.lat_min}")
print(f"lat_max: {fieldset.lat_max}")
print(f"lon_min: {fieldset.lon_min}")
print(f"lon_max: {fieldset.lon_max}")

Fieldset boundaries:
lat_min: 25.0
lat_max: 47.02000045776367
lon_min: -140.0
lon_max: -110.05999755859375


In [26]:
exP.add_perpendicular_currents_to_fieldset(fieldset)

IndexError: invalid index to scalar variable.